# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Embark on a journey to master the realm of deep learning with DeepLearning.AI, a leading online platform founded by the AI guru Andrew Ng. From beginner to advanced levels, dive into courses that unravel the mysteries of neural networks, computer vision, natural language processing, and more. Gain hands-on experience, work on real-world projects, and join a vibrant community of learners. Whether you aspire to boost your career or satisfy your curiosity, DeepLearning.AI equips you with the skills to thrive in the AI-driven world. Seize the opportunity to revolutionize your understanding of artificial intelligence today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Embark on a journey to master the realm of deep learning with DeepLearning.AI, a leading online platform founded by the AI guru Andrew Ng. From beginner to advanced levels, dive into courses that unravel the mysteries of neural networks, computer vision, natural language processing, and more. Gain hands-on experience, work on real-world projects, and join a vibrant community of learners. Whether you aspire to boost your career or satisfy your curiosity, DeepLearning.AI equips you with the skills to thrive in the AI-driven world. Seize the opportunity to revolutionize your understanding of artificial intelligence today!

---------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Embark on a journey to master the realm of deep learning with DeepLearning.AI, a leading online platform founded by the AI guru Andrew Ng. From beginner to advanced levels, dive into courses that unravel the mysteries of neural networks, computer vision, natural language processing, and more. Gain hands-on experience, work on real-world projects, and join a vibrant community of learners. Whether you aspire to boost your career or satisfy your curiosity, DeepLearning.AI equips you with the skills to thrive in the AI-driven world. Seize the opportunity to revolutionize your understanding of artificial intelligence today!

---------------------------------------------


--------------------------------------------------------------------------------
Critic (to Writer):

Feedback aggregation:
- The SEO Specialist reviewers all noted that the suggestions provided have been implemented, including incorporating target keywords, creating a clear call-to-action, and including a meta description to improve SEO.

Final suggestion on the writing:
The implementation of the SEO suggestions seems to have been successful based on the reviewers' feedback. Continuing to monitor and adjust SEO strategies based on performance data will be crucial for long-term success. Consider expanding the scope of optimization efforts beyond just keywords to ensure a comprehensive SEO strategy. Keep up the good work in ensuring the content is optimized for search engines.

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Embark on a transformative journey into de

## Get the summary

In [15]:
print(res.summary)

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Embark on a transformative journey into deep learning expertise with DeepLearning.AI, crafted by AI luminary Andrew Ng. From foundational to advanced courses, explore neural networks, computer vision, and natural language processing. Engage in practical projects, tap into a vibrant learning community. Whether you aspire career growth or knowledge, DeepLearning.AI equips you for the AI era. Seize the chance to amplify your AI proficiency today!
